In [1]:
import torch
import numpy as np
import os
import sys
sys.path.append('../code/')
from utils_ICP import sinkhorn,chamfer,draw_samples,rotation_matrix,Y_to_X
from ICP import ICP_fast,procruste

if torch.cuda.is_available():
    device='cuda'
    dtype = torch.cuda.DoubleTensor
else:
    device='cpu'
    dtype=torch.DoubleTensor
print(device)

torch.set_default_tensor_type(torch.DoubleTensor)

#Loading Data

#From .png
#xt=10*draw_samples("./data/ICP/density_b.png", n=500).numpy()

#From .pts
#xt=np.loadtxt('./data/ICP/mumble_sitting_3000.pts', comments=("version:", "n_points:", "{", "}"))
xt=np.loadtxt('./data/ICP//WitchCastle_150000.pts', comments=("version:", "n_points:", "{", "}"))

n,dim=xt.shape

Y = torch.from_numpy(xt)
Y-=torch.mean(Y,axis=0)

n_seed=50
max_iter=200

method=['swgg','nearest']
nb_proj=[100,0]


method_sinkhorn=np.zeros((n_seed,len(method)))
method_frobenius=np.zeros((n_seed,len(method)))
method_chamfer=np.zeros((n_seed,len(method)))
method_nb_ite=np.zeros((n_seed,len(method)))
method_time=np.zeros((n_seed,len(method)))

for i in range(n_seed):
    print(i,end=' ')
    X=Y_to_X(Y,trans=5,eps=.1)
    R_real,t_real=procruste(X, Y)
    for j,meth in enumerate(method):
        X_f,time,nb,R_f,t_f=ICP_fast(X,Y,pairs=meth, max_iter=max_iter, tol=1e-2,n_proj=nb_proj[j])
        method_chamfer[i,j]= chamfer(X_f,Y)
        method_sinkhorn[i,j]=sinkhorn(X_f.to(device),Y.to(device))
        method_frobenius[i,j]=torch.norm(R_real-R_f)**2+torch.linalg.norm(t_real-t_f)**2
        method_time[i,j]=time
        method_nb_ite[i,j]=nb

cuda
0 

/home/gui/.local/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 

In [2]:
print(np.mean(method_chamfer,0))

[8.51834221e-04 6.90115604e+00]


In [3]:
print(np.mean(method_frobenius,0))

[ 3.83848208 21.72730826]


In [4]:
print(np.mean(method_sinkhorn,0))

[ 6.0187961  27.01782301]


In [5]:
np.savetxt("Results/ICP/150000_fro.txt",method_frobenius)
np.savetxt("Results/ICP/150000_chamfer.txt",method_chamfer)

In [10]:
# Data can be load this way
method_frobenius=np.loadtxt('Results/ICP/3000_fro.txt')
method_chamfer=np.loadtxt('Results/ICP/3000_chamfer.txt')

In [11]:
print(np.mean(method_chamfer,0))
print(np.mean(method_frobenius,0))

[0.12818157 0.16268573 0.20371273]
[ 4.10944076  6.02722562 25.84836751]
